In [ ]:
import MLJ
using StatsPlots

In [ ]:
include("PreprocessAerosolData.jl")
include("ActivationEmulatorModels.jl")

In [ ]:
X_train, Y_train = read_aerosol_dataset("datasets/2modal_dataset2_train.csv")

In [ ]:
pipeline = preprocess_aerosol_data |> Standardizer() |> MLJ.TransformedTargetModel(MyGPRegressor(), transformer = Standardizer())
mach = MLJ.machine(pipeline, X_train, Y_train)

In [ ]:
MLJ.fit!(mach, verbosity = 2)

In [ ]:
MLJ.save("emulators/2modal_gp_machine1.jls", mach)